In [36]:
import string
import nltk
from nltk.corpus import stopwords
#nltk.download() # only need to download for once.
import json
from pprint import pprint

In [37]:
def clean_up_tweet(tweet):
    
    # remove punctuations.
    for p in string.punctuation:
        tweet = tweet.replace(p,'')
    
    tweet = tweet.split(' ')
    
    # remove stop words.
    stopWords = set(stopwords.words('english'))
    filtered = []
    for w in tweet:
        if w not in stopWords:
            filtered.append(w)
     
    # remove redundance.
    tweet_set = set(filtered)

    return tweet_set


In [38]:
# find the intersection of two tweets

def intersection(tweet1, tweet2):

    counter = 0
    for word1 in tweet1:
        for word2 in tweet2:
            if(word1 == word2):
                counter += 1
            
    return counter

In [39]:
def union(tweet1, tweet2):
    return len(tweet1) + len(tweet2) - intersection(tweet1,tweet2)

In [60]:
def jaccard_distance(tweet1, tweet2):
    return 1 - (intersection(tweet1, tweet2)/union(tweet1, tweet2))

In [64]:
def dump_tweets_to_dict(filename):
    
    # make a new dictionary to store tweets: 
    # id:text -> 323906397609791488: "RT @ItsJennaMarbles: Re..."
    tweet_dict = {}
    
    # open json file to read lines.
    with open(filename,'r') as f:
        for line in f:
            json_data = json.loads(line)
            tweet_dict[json_data.get('id')] = json_data.get('text')
   
    return tweet_dict    

dic = dump_tweets_to_dict('Tweets.json')
print(dic.get(323906483584655360))
        

RT @kustoo: RT @RedCross: Thanks to generosity of volunteer blood donors there is currently enough blood on the shelves to meet demand. #BostonMarathon


In [59]:
t1 = 'it is weather is scary is is.'
t2 = 'the weather is nice is is!'

tweet1 = clean_up_tweet(t1)
tweet2 = clean_up_tweet(t2)

# open json file to read lines.
with open('Tweets.json','r') as f:
    for line in f:
        json_data = json.loads(line)
        print(json_data.get('id'))
        print('\n')
        
        
f.closed

323906397609791488


323906397618196483


323906397735641088


323906397853073410


323906397962121216


323906398012461057


323906398230544385


323906398314438656


323906398352195585


323906398423490561


323906398176030720


323906398826164225


323906398993932289


323906399149109248


323906399295926273


323906399300100096


323906482489921537


323906484629032960


323906483584655360


323906485249789952


323906567294562306


323906650987692034


323906651209994241


323906653651079168


323906657333682176


323906656318676993


323907087551836160


323907258301939713


323907771256938496


323908455545049088


323908795254312962


323908795396943872


323909308188344320


323910330315075584


323910330457669633


323911097818173440


323911610236293120


323913403460636673


323913403951357953


323913746231750656


323915000567697409


323915624835342337


323916051614138368


323916814490939393


323917413789868034


323918009901150208


323919716039155712


3239201464544

True